In [8]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from function import lineal, sigmoide, derivate_sigmoide
import os
import math

In [9]:
#Inicializamos los pesos de la Capa 1
w = np.random.random(500)
w11 = w[:100]
w12 = w[100:200]
w13 = w[200:300]
w14 = w[300:400]
w15 = w[400:500]
W1=[w11,w12,w13,w14,w15]

#Inicializamos los pesos de la Capa 2
w = np.random.random(25)
w21 = w[:5]
w22 = w[5:10]
w23 = w[10:15]
w24 = w[15:20]
w25 = w[20:25]
W2 = [w21,w22,w23,w24,w25] 

#Incializamos los pesos de la Capa 3
w = np.random.random(15)
w31 = w[:5]
w32 = w[5:10]
w33 = w[10:15]
W3 = [w31,w32,w33]

#Bias
b = random.random()

#Factor de aprendizaje
alfa = random.random()

#Codificacion de clases
#b = 100
#d = 010
#f = 001
Ye =[{'b':[1,0,0]},{'d':[0,1,0]},{'f':[0,0,1]}]

In [10]:
data = (pd.read_csv(os.path.join(os.path.abspath(''),"data","distorsionadas",'100','letras.csv'),sep=';',header=None)).to_numpy()
#Convertimos el dataframe en array numpy
data_train = data[:int(len(data)*0.8)]
data_test = data[int(len(data)*0.8)+1:int(len(data)*0.8)+int(len(data)*0.15)]
data_validation = data[int(len(data)*0.8)+int(len(data)*0.15)+1:99]

In [11]:
#Dividiendo data train en una tupla (entrada,clase)
letras_train = []
for letra in data_train:
    x_test = letra[:100]
    y_test = letra[100:]
    letras_train.append((x_test, y_test))

In [42]:
for i in range(1):

    #Primer capa
    ys1 =[]
    for j in range(len(W1)):
        #Suma ponderada
        Z = np.dot(letras_train[i][0],W1[j])
        #Funcion de activacion
        ys1.append(lineal(Z+b))
    #print("Primer capa: ", ys1)

    #Segunda capa
    ys2 = []
    for j in range(len(W2)):
        #Suma ponderada
        Z = np.dot(ys1,W2[j])
        #Funcion de activacion
        ys2.append(sigmoide(Z+b))
    #print("2da capa: ", ys2)
    #Capa de salida
    ys3 = []
    for j in range(len(W3)):
        #Suma ponderada
        Z = np.dot(ys2,W3[j])
        #Funcion de activacion
        ys3.append(sigmoide(Z+b))
    #print("Capa salida: ", ys3)
    #!!-----Calculo del error-----!!!
    error = []
    for j in range(len(ys3)):
        # Salida espera - Salida obtenida
        error.append(letras_train[i][1][j] - ys3[j])
    #!!-------Backpropagation-----!!
    #Actualizacion W3
    -(ye-a(z))
    new_weights = []
    ys2 = np.array(ys2)
    ys3 = np.array(ys3)
    for j in range (len(W3)):
        new_weights.append(W3[j]+alfa*derivate_sigmoide(ys3[j])*error[j]*ys2)
    print("Pesos anteriores W3: ", W3)
    print("\n")
    print("Nuevos pesos de W3: ",new_weights)

Pesos anteriores W3:  [array([0.34454629, 0.3475878 , 0.73239798, 0.84657507, 0.06655497]), array([0.67860019, 0.64487817, 0.95971476, 0.24966497, 0.31261677]), array([0.79082731, 0.34903601, 0.53398945, 0.40589658, 0.47137842])]


Nuevos pesos de W3:  [array([0.32528449, 0.32832599, 0.71313618, 0.82731326, 0.04729317]), array([0.67894258, 0.64522056, 0.96005715, 0.25000736, 0.31295916]), array([0.77486226, 0.33307096, 0.51802439, 0.38993152, 0.45541337])]
